In [11]:
import requests as re
import json
import pandas as pd
import time
import numpy as np

In [12]:
url = "http://comtrade.un.org"
countries = np.array([  4,   8,  12,  20,  24,  28,  31,  32,  36,  40,  44,  48,
        50,  51,  52,  58,  60,  64,  68,  70,  76,  86,  84,  90,  92,
        96, 100, 104, 108, 116, 112, 120, 124, 132, 136, 140, 144, 148,
       152, 156, 162, 166, 170, 174, 178, 180, 184, 188, 191, 192, 196,
       200, 204, 208, 212, 214, 218, 222, 226, 230, 233, 234, 238, 242,
       246, 251, 254, 262, 258, 260, 266, 268, 270, 276, 288, 292, 296,
       300, 304, 308, 312, 320, 324, 328, 332, 340, 344, 348, 352, 360,
       364, 368, 372, 376, 380, 384, 388, 398, 392, 400, 404, 408, 410,
       414, 417, 418, 422, 428, 430, 440, 446, 450, 454, 458, 466, 462,
       470, 474, 478, 480, 484, 490, 496, 498, 500, 504, 508, 512, 520,
       524, 527, 528, 530, 533, 540, 548, 554, 558, 562, 566, 570, 574,
       579, 583, 584, 585, 586, 591, 598, 600, 604, 608, 616, 624, 620,
       634, 638, 642, 643, 646, 654, 659, 660, 662, 666, 670, 678, 682,
       686, 690, 694, 699, 702, 704, 705, 706, 711, 716, 724, 732, 736,
       740, 752, 757, 760, 762, 764, 768, 772, 776, 780, 784, 788, 792,
       795, 796, 798, 800, 804, 818, 826, 834, 854, 860, 858, 862, 876,
       882, 887, 891, 899, 894, 203, 231, 434, 612, 703, 807, 232, 839,
        72, 175, 275, 334, 336, 426, 442, 516, 626, 674, 748, 710,  56,
       499, 688, 534, 531, 729, 728])
numy = lambda x: f'0{x}' if x<10 else f'{x}'
products = ['%2C'.join([numy(i) for i in range(j*12+1,j*12+13)]) for j in range(2)]
years = ['%2C'.join([str(year) for year in range(j*3+2015,j*3+2018)]) for j in range(2)]

In [13]:
total_requests = 0
start = time.time()

In [39]:
all_records = []
# TODO: sometimes the api answer is empty, but if we try with a smaller range of years, it is non-empty. FIX
for country in countries:
    if country <= 246: continue
    country_rec = []
    for year in ['%2C'.join(str(year) for year in range(2001+k*5,2006+k*5)) for k in range(4)]:#years:
        for product in products:
            # Get data
            result = re.get(url+f"/api/get?max=100000&type=C&freq=A&px=HS&ps={year}&r={country}&p=all&rg=1&cc={product}")
            data = result.json()['dataset']
            time.sleep(1)

            # Validate obtained data is less than the api limit
            print(f'entries: {len(data)}')
            if len(data) == 10000 or len(data) == 0:
                print(f'Data may not be complete for code {country}')

            # Add new entries
            all_records.extend(data)
            country_rec.extend(data)
            
            # Account for the request just performed
            total_requests += 1
            now = time.time()
            
            # Reset everything if more than an hour has elapsed
            if now - start > 3600:
                start = now
                total_requests = 0

            # If the number of requests
            if total_requests > 99: #and (now-start < 3600): Alredy accounted for in the if statement above
                time.sleep(3600-now + start + 10)
                start = time.time()
                total_requests = 0

    # Save the country data
    df = pd.DataFrame(country_rec)
    print(f'Writing to memory country with code {country}')
    df.to_csv(f'{country}_2001-2020.csv')

2001%2C2002%2C2003%2C2004%2C2005
entries: 1887
entries: 2193
2006%2C2007%2C2008%2C2009%2C2010
entries: 1876
entries: 2404
2011%2C2012%2C2013%2C2014%2C2015
entries: 2125
entries: 2627
2016%2C2017%2C2018%2C2019%2C2020
entries: 2245
entries: 0
Data may not be complete for code 76
Writing to memory country with code 76


In [ ]:
result

In [ ]:
countries[186:]

In [20]:
total_requests

106

In [18]:
['%2C'.join(str(year) for year in range(2001+k*5,2006+k*5)) for k in range(4)]

['2001%2C2002%2C2003%2C2004%2C2005',
 '2006%2C2007%2C2008%2C2009%2C2010',
 '2011%2C2012%2C2013%2C2014%2C2015',
 '2016%2C2017%2C2018%2C2019%2C2020']

In [17]:
['%2C'.join([numy(i) for i in range(j*12+1,j*12+13)]) for j in range(2)]

['01%2C02%2C03%2C04%2C05%2C06%2C07%2C08%2C09%2C10%2C11%2C12',
 '13%2C14%2C15%2C16%2C17%2C18%2C19%2C20%2C21%2C22%2C23%2C24']

In [53]:
year = '2016%2C2017%2C2018%2C2019%2C2020'
product = ['%2C'.join([numy(i) for i in range(j*12+1,j*12+13)]) for j in range(2)][1]
country = 76 #brazil
result = re.get(url+f"/api/get?max=100000&type=C&freq=A&px=HS&ps={year}&r={country}&p=all&rg=1&cc={product}")
result

<Response [200]>

In [54]:
data = result.json()['dataset']

In [55]:
years = set([d['yr'] for d in data])
years

set()

In [56]:
len(data)

0